<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [8]:
# ## install finrl library
# !pip install wrds
# !pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

<a id='1.4'></a>
## 2.4. Create Folders

In [10]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [11]:
# print(config_tickers.DOW_30_TICKER)

In [12]:
# dp = YahooFinanceProcessor()
# df = dp.download_data(start_date = '2008-01-01',
#                      end_date = '2021-10-31',
#                      ticker_list = config_tickers.DOW_30_TICKER, time_interval='1D')

In [13]:
df = pd.read_csv("/content/tickers.csv")

In [14]:
tickers = ["^TNX", "XU030.IS"]
# tickers = ["XU030.IS"]

In [15]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df = df.sort_values(['date','tic'],ignore_index=True)
# Filter the DataFrame to keep only rows where the date is common for all tickers
df = df[df.groupby('date')['tic'].transform('nunique') == len(tickers)]

In [16]:
df.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day
0,1,2000-01-04,19368.000000,22593.099609,19368.000000,22406.000000,0,XU030.IS,0
1,2,2000-01-04,6.530000,6.548000,6.485000,6.485000,0,^TNX,1
2,3,2000-01-05,22406.000000,22762.800781,20585.199219,21475.099609,0,XU030.IS,1
3,4,2000-01-05,6.521000,6.599000,6.508000,6.599000,0,^TNX,2
4,5,2000-01-06,21475.099609,22006.800781,19998.500000,20136.500000,0,XU030.IS,2


In [17]:
df.shape

(11454, 9)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [18]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [19]:
df.shape

(11454, 17)

In [20]:
df.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,1,2000-01-04,19368.000000,22593.099609,19368.000000,22406.000000,0,XU030.IS,0,0.000000,23257.041762,20624.057847,0.0,66.666667,100.00000,22406.000000,22406.000000
5727,2,2000-01-04,6.530000,6.548000,6.485000,6.485000,0,^TNX,1,0.000000,23257.041762,20624.057847,0.0,66.666667,100.00000,6.485000,6.485000
1,3,2000-01-05,22406.000000,22762.800781,20585.199219,21475.099609,0,XU030.IS,1,-20.885586,23257.041762,20624.057847,0.0,66.666667,100.00000,21940.549805,21940.549805
5728,4,2000-01-05,6.521000,6.599000,6.508000,6.599000,0,^TNX,2,0.002558,6.703220,6.380780,100.0,66.666667,100.00000,6.542000,6.542000
2,5,2000-01-06,21475.099609,22006.800781,19998.500000,20136.500000,0,XU030.IS,2,-68.122275,23620.873919,19057.525820,0.0,-100.000000,57.40766,21339.199870,21339.199870


## Add covariance matrix as states

In [21]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


In [22]:
df.shape

(10950, 19)

In [23]:
df.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,525,2001-01-22,14504.299805,14620.900391,13992.900391,14110.500000,0,XU030.IS,260,341.470746,15154.710888,10723.518898,51.169720,124.276097,15.770246,12467.549967,14153.008301,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
1,526,2001-01-22,5.172000,5.233000,5.172000,5.221000,0,^TNX,265,-0.050137,5.299270,4.876930,45.262048,10.406064,18.401984,5.179433,5.436333,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
2,527,2001-01-23,14110.500000,14110.500000,13470.400391,13846.799805,0,XU030.IS,261,332.969260,15232.265793,10838.084011,50.260409,87.268167,6.394814,12600.846615,14103.358285,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
3,528,2001-01-23,5.217000,5.279000,5.172000,5.274000,0,^TNX,266,-0.035650,5.316921,4.869079,47.104754,33.610920,22.623918,5.171467,5.430033,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
4,529,2001-01-24,13846.799805,14452.500000,13608.900391,13877.299805,0,XU030.IS,262,324.947078,15272.316781,11021.103043,50.365943,91.691844,11.414324,12757.343262,14053.948275,"[[0.0015422700744588604, -3.862360111952201e-0...",tic XU030.IS ^TNX date ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [24]:
train = data_split(df, '2000-01-01','2018-04-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)

In [25]:
train.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,525,2001-01-22,14504.299805,14620.900391,13992.900391,14110.500000,0,XU030.IS,260,341.470746,15154.710888,10723.518898,51.169720,124.276097,15.770246,12467.549967,14153.008301,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
0,526,2001-01-22,5.172000,5.233000,5.172000,5.221000,0,^TNX,265,-0.050137,5.299270,4.876930,45.262048,10.406064,18.401984,5.179433,5.436333,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
1,527,2001-01-23,14110.500000,14110.500000,13470.400391,13846.799805,0,XU030.IS,261,332.969260,15232.265793,10838.084011,50.260409,87.268167,6.394814,12600.846615,14103.358285,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
1,528,2001-01-23,5.217000,5.279000,5.172000,5.274000,0,^TNX,266,-0.035650,5.316921,4.869079,47.104754,33.610920,22.623918,5.171467,5.430033,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
2,529,2001-01-24,13846.799805,14452.500000,13608.900391,13877.299805,0,XU030.IS,262,324.947078,15272.316781,11021.103043,50.365943,91.691844,11.414324,12757.343262,14053.948275,"[[0.0015422700744588604, -3.862360111952201e-0...",tic XU030.IS ^TNX date ...


## Environment for Portfolio Allocation


In [26]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [27]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 2, State Space: 2


In [28]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [29]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [30]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [31]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [32]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 293       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -2.83     |
|    explained_variance | 5.96e-08  |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 1e+07     |
|    reward             | 948949.06 |
|    std                | 0.996     |
|    value_loss         | 1.02e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 331       |
|    iterations         | 200       |
|    time_elapsed       | 3         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -2.83     |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0002    |
|    n_updat

In [33]:
trained_a2c.save('/content/trained_models/trained_a2c.zip')

### Model 2: **PPO**


In [34]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [35]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=80000)

----------------------------------
| time/              |           |
|    fps             | 487       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 2318809.5 |
----------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 351       |
|    iterations           | 2         |
|    time_elapsed         | 11        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.84     |
|    explained_variance   | 0         |
|    learning_rate        | 0.0001    |
|    loss                 | 4.47e+14  |
|    n_updates            | 10        |
|    policy_gradient_loss | -1.2e-07  |
|    reward               | 3873835.8 |
|    

In [36]:
trained_ppo.save('/content/trained_models/trained_ppo.zip')

### Model 3: **DDPG**


In [37]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [38]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:1512632.5250064014
Sharpe:  0.22704622968376184
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 49        |
|    time_elapsed    | 337       |
|    total_timesteps | 16648     |
| train/             |           |
|    actor_loss      | -4.15e+07 |
|    critic_loss     | 2.94e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 12486     |
|    reward          | 1551810.1 |
----------------------------------
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin

In [39]:
trained_ddpg.save('/content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [40]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [41]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:1689793.9387617712
Sharpe:  0.25231184707634785
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 36        |
|    time_elapsed    | 457       |
|    total_timesteps | 16648     |
| train/             |           |
|    actor_loss      | -6.46e+07 |
|    critic_loss     | 8.02e+11  |
|    ent_coef        | 14.1      |
|    ent_coef_loss   | -49.3     |
|    learning_rate   | 0.0003    |
|    n_updates       | 16547     |
|    reward          | 1551810.1 |
----------------------------------
begin_total_asset:1000000
end_total_asset:1551810.1452385047
Sharpe:  0.23235582691794376
begin_total_asset:1000000

In [42]:
trained_sac.save('/content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [43]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [44]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

begin_total_asset:1000000
end_total_asset:5818657.490336113
Sharpe:  0.5336252462119315
begin_total_asset:1000000
end_total_asset:6036877.144936982
Sharpe:  0.5355566028067438
begin_total_asset:1000000
end_total_asset:6036877.144936982
Sharpe:  0.5355566028067438
begin_total_asset:1000000
end_total_asset:6036877.144936982
Sharpe:  0.5355566028067438
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 59        |
|    time_elapsed    | 277       |
|    total_timesteps | 16648     |
| train/             |           |
|    actor_loss      | -6.18e+07 |
|    critic_loss     | 8.99e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 12486     |
|    reward          | 6036877.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:6036877.144936982
Sharpe:  0.5355566028067438
begin_total_asset:1000000
end_total_asset:6036877.144936982
Sharpe:  0.5355566028067438
begin_total_asset

In [45]:
trained_td3.save('/content/trained_models/trained_td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [46]:
trade = data_split(df,'2018-04-01', '2023-01-01')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)


In [47]:
trade.shape

(2290, 19)

In [48]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1109077.9874204672
Sharpe:  0.41420343956264183
hit end!


In [49]:
df_daily_return.head()

,date,daily_return
0,2018-04-02,0.000000
1,2018-04-03,0.006474
2,2018-04-04,-0.002619
3,2018-04-05,0.012564
4,2018-04-06,-0.008647


In [50]:
df_daily_return.to_csv('df_daily_return.csv')

In [51]:
df_actions.head()

,XU030.IS,^TNX
date,,
2018-04-02,0.500000,0.500000
2018-04-03,0.731059,0.268941
2018-04-04,0.731059,0.268941
2018-04-05,0.731059,0.268941
2018-04-06,0.731059,0.268941


In [52]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [53]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func( returns=DRL_strat,
                              factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")

In [54]:
print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


Annual return            0.023047
Cumulative returns       0.109078
Annual volatility        0.442600
Sharpe ratio             0.414203
Calmar ratio             0.028057
Stability                0.213429
Max drawdown            -0.821443
Omega ratio              1.115752
Sortino ratio            0.469817
Skew                   -14.829788
Kurtosis               377.884907
Tail ratio               0.988815
Daily value at risk     -0.055035
Alpha                    0.000000
Beta                     1.000000
dtype: float64

In [55]:
# baseline_df = pd.read_csv("/content/baseline_data.csv")
# baseline_df['date'] = pd.to_datetime(baseline_df['date'])
# baseline_df['date'] = baseline_df['date'].dt.strftime('%Y-%m-%d')
# baseline_df = baseline_df.sort_values(['date','tic'],ignore_index=True)
# # Filter the DataFrame to keep only rows where the date is common for all tickers
# baseline_df.head()

,date,open,high,low,close,volume,tic,day
0,2018-04-02,24076.599609,24123.800781,23344.519531,23644.189453,509820000,^DJI,0
1,2018-04-03,23698.330078,24044.349609,23664.330078,24033.359375,394130000,^DJI,1
2,2018-04-04,23654.150391,24308.960938,23523.160156,24264.300781,388540000,^DJI,2
3,2018-04-05,24313.910156,24622.259766,24313.910156,24505.220703,347790000,^DJI,3
4,2018-04-06,24373.599609,24434.400391,23738.199219,23932.759766,395990000,^DJI,4


In [56]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])


stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========
Annual return          0.073651
Cumulative returns     0.401919
Annual volatility      0.217929
Sharpe ratio           0.435969
Calmar ratio           0.198595
Stability              0.679414
Max drawdown          -0.370862
Omega ratio            1.091783
Sortino ratio          0.603538
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.899592
Daily value at risk   -0.027079
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [57]:
baseline_df = pd.read_csv("/content/baseline_data.csv")
baseline_df['date'] = pd.to_datetime(baseline_df['date'])
baseline_df['date'] = baseline_df['date'].dt.strftime('%Y-%m-%d')
baseline_df = baseline_df.sort_values(['date','tic'],ignore_index=True)
# Filter the DataFrame to keep only rows where the date is common for all tickers
baseline_df.head()

,date,open,high,low,close,volume,tic,day
0,2018-04-02,24076.599609,24123.800781,23344.519531,23644.189453,509820000,^DJI,0
1,2018-04-03,23698.330078,24044.349609,23664.330078,24033.359375,394130000,^DJI,1
2,2018-04-04,23654.150391,24308.960938,23523.160156,24264.300781,388540000,^DJI,2
3,2018-04-05,24313.910156,24622.259766,24313.910156,24505.220703,347790000,^DJI,3
4,2018-04-06,24373.599609,24434.400391,23738.199219,23932.759766,395990000,^DJI,4


In [58]:
baseline_df.tail()

,date,open,high,low,close,volume,tic,day
1193,2022-12-23,32961.058594,33226.140625,32814.019531,33203.929688,221050000,^DJI,1193
1194,2022-12-27,33224.230469,33387.718750,33069.578125,33241.558594,246010000,^DJI,1194
1195,2022-12-28,33264.761719,33379.550781,32869.148438,32875.710938,252260000,^DJI,1195
1196,2022-12-29,33021.429688,33293.421875,33020.351562,33220.800781,243060000,^DJI,1196
1197,2022-12-30,33121.609375,33152.550781,32847.820312,33147.250000,295500000,^DJI,1197


In [59]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^DJI', start="2018-04-01", end='2023-01-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (1198, 8)


AttributeError: ignored

## Min-Variance Portfolio Allocation

In [ ]:
%pip install PyPortfolioOpt

In [60]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [61]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [62]:
df.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,525,2001-01-22,14504.299805,14620.900391,13992.900391,14110.500000,0,XU030.IS,260,341.470746,15154.710888,10723.518898,51.169720,124.276097,15.770246,12467.549967,14153.008301,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
1,526,2001-01-22,5.172000,5.233000,5.172000,5.221000,0,^TNX,265,-0.050137,5.299270,4.876930,45.262048,10.406064,18.401984,5.179433,5.436333,"[[0.001562591220045976, -3.92435968182094e-05]...",tic XU030.IS ^TNX date ...
2,527,2001-01-23,14110.500000,14110.500000,13470.400391,13846.799805,0,XU030.IS,261,332.969260,15232.265793,10838.084011,50.260409,87.268167,6.394814,12600.846615,14103.358285,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
3,528,2001-01-23,5.217000,5.279000,5.172000,5.274000,0,^TNX,266,-0.035650,5.316921,4.869079,47.104754,33.610920,22.623918,5.171467,5.430033,"[[0.001557292288070286, -3.7044574924766546e-0...",tic XU030.IS ^TNX date ...
4,529,2001-01-24,13846.799805,14452.500000,13608.900391,13877.299805,0,XU030.IS,262,324.947078,15272.316781,11021.103043,50.365943,91.691844,11.414324,12757.343262,14053.948275,"[[0.0015422700744588604, -3.862360111952201e-0...",tic XU030.IS ^TNX date ...


In [63]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

portfolio=portfolio.T
portfolio.columns = ['account_value']

OptimizationError: ignored

In [ ]:
portfolio.head()

In [64]:
a2c_cumpod =(df_daily_return.daily_return+1).cumprod()-1

In [65]:
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1

AttributeError: ignored

In [66]:
dji_cumpod =(baseline_returns+1).cumprod()-1

## Plotly: DRL, Min-Variance, DJIA

In [67]:
%pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 31.2 MB/s eta 0:00:00


In [68]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [69]:
time_ind = pd.Series(df_daily_return.date)

In [70]:
trace0_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod, mode = 'lines', name = 'A2C (Portfolio Allocation)')

trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')
#trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
#trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
#trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

#trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

#trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
#trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

NameError: ignored

In [ ]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)



fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2

    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date',
       'tick0': time_ind[0],
        'tickmode': 'linear',
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()